In [50]:
import requests, pickle, os, re
import pandas as pd
from bs4 import BeautifulSoup as bs

# Overall Teams 
* Team name team id
    * http://stats.ncaa.org/team/inst_team_list?academic_year=2018&conf_id=-1&division=11&sport_code=MFB
* Team history http://stats.ncaa.org/teams/history/MFB/726

# Files format
- team_year_schedule 
- team_year_roster
- team_year_stats_type 
- hc

# Flow 
<br>
Teams(year)
<ul>
    <li>Schedule/Results</li>
    <li>Team Stats</li>
    <li><font color="red">Head coach</font></li> 
    <li>Team History</li> 
</ul>
Roster
<ul> 
    <li>Roster</li>
</ul>
Team Statistics
<ul> 
    <li> Rushing </li> 
    <li> Passing </li> 
    <li> Receiving </li> 
    <li> Total Offense </li>
    <li> All-purpose Yards </li> 
    <li> Scoring </li> 
    <li> Sacks </li>
    <li> Tackles </li> 
    <li> Passes Defended </li> 
    <li> Fumbles </li> 
    <li> Kicking </li> 
    <li> Punting </li> 
    <li> Punt Returns </li> 
    <li> Kickoffs and KO Returns </li> 
    <li> Redzone </li> 
    <li> Defense </li> 
    <li> Turnover Margin </li> 
    <li> Participation </li> 
</ul> 
<font color="red">Previous years</font>

    
    
    



# Pull in the all the different teams

In [2]:
years = [2016, 2017]
schoolURL = "http://stats.ncaa.org/team/inst_team_list?academic_year={year}&conf_id=-1&division=11&sport_code=MFB"
teams = {}

In [3]:
# Check to see if the file exists, if the file exists, just use the file
# if it doesn't exist query the results and then pickle it 

if os.path.isfile("teams.p"):
    print("Found a pickle of the teams...loading..")
    teams = pickle.load(open('teams.p', 'rb'))
    print("Done!")
else: 
    print("Pickle doesn't exist....finding data")
    for year in years: 
        url = schoolURL.format(year = year)
        r = requests.get(url)
        teams[year] = r.content
    print("Saving pickle...")
    pickle.dump(teams, open('teams.p', 'wb'))
    print("Done")

Found a pickle of the teams...loading..
Done!


# Loop through the teams and extract the team code and year code

In [7]:
teamURLs = {}
for year, html in teams.items(): 
    # Create the soup 
    soup = BeautifulSoup(html, 'html.parser')
    team_soup = soup.find("div", class_="css-panes")
    # Parse the soup 
    tables = team_soup.find_all("table")
    for idx, table in enumerate(tables): 
        rows = table.find_all("tr")
        for row in rows: 
            cell = row.find("td")
            url = cell.find("a").get('href').split("/")[2:4]
            team = cell.find("a").contents[0]
            # See if the team is in the dict 
            if team not in teamURLs: 
                teamURLs[team] = {}
            # See if the year is in there 
            if year not in teamURLs[team]:
                teamURLs[team][year] = {'url':[]}
            # See if the URL is in there 
            if url not in teamURLs[team][year]['url']: 
                teamURLs[team][year]['url'].append(url)

# URLS

In [95]:
base_team_url = "http://stats.ncaa.org/team/{id}/{year}"
school_history = "http://stats.ncaa.org/teams/history/MFB/{id}"
roster = "http://stats.ncaa.org/team/{id}/roster/{year}"
stats = "http://stats.ncaa.org/team/{id}/stats/{year}"
head_coach = "http://stats.ncaa.org/people/{coach}"

In [96]:
_id = teamURLs['Nebraska'][2017]['url'][0][0]
year = teamURLs['Nebraska'][2017]['url'][0][1]
url = school_history.format(id=_id, year=year)
print(url)
#r = requests.get(url.format(id=_id, year=year))

http://stats.ncaa.org/teams/history/MFB/463


# History 

In [98]:
_id = 463
r = requests.get(school_history.format(id = _id))
html = r.content 
soup = BeautifulSoup(html, "html.parser")

In [112]:
#### May need to fix this for coaches who leave during the year....
coaches_out = {}
coaches = soup.find_all("a", {'href': re.compile(r'/people/*')})
for coach in coaches: 
    name = coach.text
    _id = coach.get("href").split("/")[2]
    if _id not in coaches_out: 
        # Drop coaches who only were only around for part of a season
        if "(" not in name: 
            coaches_out[_id] = name

# Headcoach

In [123]:
tmp_coach = list(coaches_out.keys())[0]
url = head_coach.format(coach = tmp_coach)
r = requests.get(url)

In [129]:
html = r.content
soup = BeautifulSoup(html, 'html.parser')
tables = soup.find_all("table")
x = pd.read_html(soup.prettify(), flavor='bs4')
x[0].head()

,Sport,Year,Org,Division,Wins,Losses,Ties,WL%,Notes
0,MBB,1978-79,Western St.,D-II,2,25,0.0,0.074,NaN
1,MBB,1979-80,Western St.,Non-member,8,18,NaN,0.308,NaN
2,MBB,1980-81,Western St.,Non-member,12,14,NaN,0.462,NaN
3,MBB,1981-82,Western St.,Non-member,15,10,NaN,0.600,NaN
4,MFB,1997-98,Oregon St.,FBS,3,8,0.0,0.273,NaN


# Scrape the page to get the stats links

In [28]:
html = r.content
soup = BeautifulSoup(html, 'html.parser')
team_soup = soup.find("div", {"id":"stats_div"})
tables = team_soup.find_all("table")
for table in tables: 
    rows = table.find_all("tr")
    for row in rows: 
        cell = row.find_all("td")
        for c in cell: 
            try: 
                url = c.find("a").get('href')
                stat = c.find("a").contents[0]
            except: 
                url = None
                stat = c.text.replace("|", "").strip()
            print(url, stat)

In [50]:
r = requests.get("http://stats.ncaa.org/teams/history/MFB/726")

In [51]:
tables = pd.read_html(r.content)

# Rivals rankings

In [22]:
rivals = "https://n.rivals.com/team_rankings/2018/all-teams/football"

In [23]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [24]:
driver = webdriver.Firefox()
driver.get(rivals)
elem = driver.find_element_by_xpath("//*")
source_code = elem.get_attribute("outerHTML")
tables = pd.read_html(source_code)
tables[0].head()

# 247 Rankings

In [43]:
_247 = "https://247sports.com/Season/2017-Football/CompositeTeamRankings"

In [47]:
r = requests.get(_247)

In [48]:
r.content

b' <!DOCTYPE html> <html xmlns="http://www.w3.org/1999/xhtml" data-template="responsive" class="twofourseven-site"> <head> <meta charset="utf-8"> <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"> <meta name="viewport" content="width=device-width, initial-scale=1"> <meta name="format-detection" content="telephone=no"/> <meta name="application-name" content="247Sports" /> <meta name="msapplication-TileColor" content="#005695" /> <meta name="msapplication-TileImage" content="https://assets.247sports.com/Content/Img/logo_win8.png" /> <title>  2017 Football Team Rankings </title> <link rel="apple-touch-icon" href="https://assets.247sports.com/Content/Img/247touch.png" /> <link href="https://assets.247sports.com/favicon.ico" type="image/x-icon" rel="icon" /> <link href="https://assets.247sports.com/favicon.ico" type="image/x-icon" rel="shortcut icon" /> <meta property="fb:app_id" content="1523983947844395"/>  <meta property="fb:pages" content="130373190333432" />  <!--[if lt IE 

In [45]:
driver.get(_247)
elem = driver.find_element_by_xpath("//*")
source_code = elem.get_attribute("outerHTML")
# tables = pd.read_html(source_code)
# tables[0].head()

In [49]:
driver.close()

# Requests

In [55]:
r = requests.get("http://stats.ncaa.org/team/721/11520")

In [56]:
r.content

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">\n<html>\n  <head>\n    <title>NCAA Statistics</title>\n    <link href="/assets/application-b57f7c1b3aa08f4d0d4dc12332548eb4.css" media="all" rel="stylesheet" type="text/css" />\n    <script src="/assets/application-469a9d0f7635b73331b409d057f4ec1b.js" type="text/javascript"></script>\n    <meta content="authenticity_token" name="csrf-param" />\n<meta content="iMl+CM2teSRaYgdTgrWSbmfWEz9aTdOFC80Y070EDdY=" name="csrf-token" />\n    <style>\ntfoot tr:nth-child(n+2){\n  border: initial;\n  font-weight: initial;\n}\n\ntfoot tr:nth-child(n+2) td{\n  border: initial;\n  font-weight: initial;\n}\n    </style>\n\n  </head>\n  \n  <script>\n\n  function changeSport(field){\n    var el = $(field);\n    var tmList = el.parent().next().find(\'.new-team-year\')[0];\n    if (tmList == null){\n      tmList = el.nextAll(\'.new-team-year\')[0];\n    }\n    $.ajax({\n      url: "/game_sport_yea

In [57]:
tables = pd.read_html(r.content)

In [69]:
tables[1].head(15)

,0,1,2
0,Schedule/Results,NaN,NaN
1,Date,Opponent,Result
2,08/31/2013,Colgate,W 38 - 13
3,09/07/2013,Utah St.,L 20 - 52
4,09/13/2013,@ Boise St.,L 20 - 42
5,09/21/2013,Wyoming,L 23 - 56
6,09/28/2013,@ Nevada,L 42 - 45
7,10/05/2013,@ Navy,L 10 - 28
8,10/10/2013,San Diego St.,L 20 - 27
9,10/26/2013,Notre Dame,L 10 - 45


In [71]:
tables[2].head()

,0,1,2
0,Team Stats - Through games 01/06/2014,NaN,NaN
1,Stat,Rank,Value
2,Total Offense,91,367.6
3,Rushing Offense,12,262.7
4,Passing Offense,121,104.9


http://stats.ncaa.org/people/31099?sport_code=MFB

http://stats.ncaa.org/teams/history/MFB/721

http://stats.ncaa.org/team/721/roster/11520

http://stats.ncaa.org/team/721/stats/11520

http://stats.ncaa.org/team/721/stats?id=11520&year_stat_category_id=10401

In [ ]:
people = ".*/\d+\?sport_code\=MFB"
history = ".*\/teams\/history\/MFB\/\d+"
roster = ".*\/team\/\d+\/roster\/\d+"
stats = ".*\/team\/\d+\/stats\/\d+"